In [ ]:
# Import Libraries.

library("Lamian")
library("Seurat")
library("dplyr")
library("ggplot2")
library("RColorBrewer")
library("scales")

In [ ]:
### Load Data.

setwd("/folder/")
Seurat <- readRDS("Seurat.rds")

In [ ]:
### Εxtract Νormalized Εxpression.

norm_counts <- as.matrix(GetAssayData(object = Seurat, assay = "RNA", slot = "data"))

In [ ]:
### Remove Genes with Zero Expression Across all Cells.

gene_sums <- rowSums(norm_counts)
keep_genes <- gene_sums > 0
norm_counts_filt <- norm_counts[keep_genes, , drop = FALSE]

In [ ]:
### PCA coordinates from Seurat.

pca_df <- as.data.frame(Embeddings(Seurat, reduction = "pca"))
pca_df$Cell_ID <- rownames(pca_df)

In [ ]:
# Build Cell-level Metadata Required by Lamian.

meta <- Seurat@meta.data %>%
  as.data.frame() %>%
  mutate(Cell_ID = rownames(.)) %>%
  select(Cell_ID, Sample_ID, Cluster_Column)

In [ ]:
res <- infer_tree_structure(
  pca = pca_df,
  expression = norm_counts_filt,
  cellanno = meta,
  origin.marker = "PDGFRA", ### Or "FRY" for Olig_1
  origin.celltype = NA,
  xlab = "PCA_1",
  ylab = "PCA_2"
)

In [ ]:
### Visualize Cluster/Trajectory.

traj_plot <- Lamian::plotmclust(res, cell_point_size = 0.5, x.lab = "PCA_1", y.lab = "PCA_2")
print(traj_plot)

In [ ]:
### Build Pseudotime DF (Numeric Pseudotime).

pseudotime_vec <- as.numeric(res$pseudotime)
pseudotime_df <- data.frame(Cell_ID = names(res$pseudotime), Pseudotime = pseudotime_vec)

In [ ]:
### Plot.

reduction_df <- as.data.frame(res$pcareduceres) %>% tibble::rownames_to_column("Cell_ID")
merged_pseudo <- dplyr::left_join(reduction_df, pseudotime_df, by = "Cell_ID")

pseudo_plot <- ggplot(merged_pseudo, aes(x = PC_1, y = PC_2, colour = Pseudotime)) +
  geom_point(size = 0.8, alpha = 0.8) +
  scale_color_gradientn(colors = c("black", "#2488F0", "#7F3F98", "#FCB31A", "#E22929")) +
  theme_minimal() +
  labs(title = "Pseudotime (Lamian)", colour = "Pseudotime")

print(pseudo_plot)

In [ ]:
### Compute Mean Pseudotime per Cluster_Column.

meta_with_pt <- meta %>% left_join(pseudotime_df, by = "Cell_ID")
pseudotime_means <- meta_with_pt %>%
  group_by(Cluster_Column) %>%
  summarise(mean_pseudotime = mean(Pseudotime, na.rm = TRUE))

print(pseudotime_means)

In [ ]:
### Evaluate uncertainty (permutation-based)

uncert_res <- evaluate_uncertainty(res, n.permute = 1000)
str(uncert_res)

In [ ]:
### Module 2: Prepare per-sample Metadata for Branch/Condition Tests

data_matrix <- uncert_res[[2]]
sample_meta <- Seurat@meta.data %>%
  as.data.frame() %>%
  distinct(Sample_ID, Cluster_Column) %>%
  mutate(Condition = ifelse(Cluster_Column %in% c("Cluster_1","Cluster_2"), 0, 1))

sample_meta <- sample_meta[match(colnames(data_matrix), sample_meta$Sample_ID), , drop = FALSE]

branch_test_res <- branchPropTest(data = data_matrix, design = sample_meta)
print(branch_test_res)

In [ ]:
### Module 3: lamian_test for trajectory-differential-expression (XDE)

design_mat <- sample_meta %>%
  select(Sample_ID, Condition) %>%
  tibble::column_to_rownames("Sample_ID")

lamian_res <- lamian_test(
  expr = norm_counts_filt,
  cellanno = meta %>% select(-Cluster_Column),
  pseudotime = res$pseudotime,
  design = design_mat,
  testvar = 1, 
  permuiter = 1000,
  test.type = "Variable",
  test.method = "permutation",
  verbose.output = TRUE
)

In [ ]:
### Extract Significant Dynamic Genes (XDE)

stat <- lamian_res$statistics
diffgene <- rownames(stat)[stat[, grepl("^fdr.*overall$", colnames(stat))] < 0.05]

In [ ]:
### Population Fit and Clustering.

lamian_res$populationFit <- getPopulationFit(lamian_res, gene = diffgene, type = "variable")
lamian_res$covariateGroupDiff <- getCovariateGroupDiff(testobj = lamian_res, gene = diffgene)
lamian_res$cluster <- clusterGene(lamian_res, gene = diffgene, type = "variable", k = 5)

In [ ]:
### Plot.

plotXDEHm(lamian_res, cellWidthTotal = 180, cellHeightTotal = 350, subsampleCell = FALSE, sep = ':.*')
plotClusterMeanAndDiff(lamian_res)